Follow Step 1 from the below link and get your Kaggle Token
https://medium.com/@yvettewu.dw/tutorial-kaggle-api-google-colaboratory-1a054a382de0
---
Upload best_model.pth to drive or runtime of colab
https://drive.google.com/file/d/1bVVZghsfU7q5lpSwGSNTSr36MIWCCId3/view?usp=sharing


In [0]:
from google.colab import files


In [0]:
!pip install -q kaggle

Upload the kaggle token downloaded.

# New Section

In [0]:
uploaded = files.upload()

In [3]:
!ls


kaggle.json  segl0w1.jpg  segl1w4.jpg  segl1w6.jpg  segl1w8.jpg
sample_data  segl1w3.jpg  segl1w5.jpg  segl1w7.jpg  segl1w9.jpg


In [0]:
!mkdir -p ~/.kaggle



In [0]:
!cp kaggle.json ~/.kaggle/



In [6]:
!kaggle datasets download -d xainano/handwrittenmathsymbols

 99% 341M/343M [00:05<00:00, 73.4MB/s]
100% 343M/343M [00:05<00:00, 64.9MB/s]


In [7]:
!unzip handwrittenmathsymbols.zip -d handwrittenmathsymbols


Streaming output truncated to the last 5000 lines.
  inflating: handwrittenmathsymbols/extracted_images/0/0_2414.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24177.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_2420.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24398.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24469.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_2462.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24690.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24761.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_2490.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_24982.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_2501.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_25053.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_25274.jpg  
  inflating: handwrittenmathsymbols/extracted_images/0/0_

In [0]:
get_ipython().system_raw("unrar x handwrittenmathsymbols/data.rar")


In [0]:
import numpy as np
import pandas as pd

In [10]:
!pip install split-folders

In [0]:
import os
import numpy as np
from tqdm import tqdm
from fastai import *
import torch
from fastai.vision import *
%matplotlib inline
from fastai.callbacks import *
import cv2
import pandas as pd
import split_folders

In [0]:
image_load_size = 64
bs = 24

In [0]:
 def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True    

SEED = 999
seed_everything(SEED)

In [14]:
split_folders.ratio('extracted_images', output="final_output_images", seed=SEED, ratio=(.8, .2))

Copying files: 375974 files [00:50, 7448.46 files/s]


In [0]:
tfms = get_transforms(do_flip=True, flip_vert=False, max_lighting=0.1, max_zoom=1.05,
                      max_warp=0.,
                      xtra_tfms=[rand_crop(), rand_zoom(1, 1.5),
                                 symmetric_warp(magnitude=(-0.2, 0.2))])

In [0]:
data = (ImageList.from_folder(path='final_output_images')
        .split_by_folder(train='train',valid='val')
        .label_from_folder()
        .transform(tfms,size = image_load_size,resize_method=ResizeMethod.SQUISH)
        .databunch(path='.',bs=bs)    
        .normalize(imagenet_stats)
)

In [0]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1., gamma=2.):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets, **kwargs):
        CE_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-CE_loss)
        F_loss = self.alpha * ((1-pt)**self.gamma) * CE_loss
        return F_loss.mean()

In [18]:
model = cnn_learner(data,models.densenet161, metrics = [accuracy,error_rate],callback_fns=[partial(SaveModelCallback, monitor='accuracy', name='best_model')])
model.loss_func = FocalLoss()
model.summary()

Downloading: "https://download.pytorch.org/models/densenet161-8d451a50.pth" to /root/.cache/torch/checkpoints/densenet161-8d451a50.pth


Sequential
Layer (type)         Output Shape         Param #    Trainable 
Conv2d               [96, 32, 32]         14,112     False     
______________________________________________________________________
BatchNorm2d          [96, 32, 32]         192        True      
______________________________________________________________________
ReLU                 [96, 32, 32]         0          False     
______________________________________________________________________
MaxPool2d            [96, 16, 16]         0          False     
______________________________________________________________________
BatchNorm2d          [96, 16, 16]         192        True      
______________________________________________________________________
ReLU                 [96, 16, 16]         0          False     
______________________________________________________________________
Conv2d               [192, 16, 16]        18,432     False     
___________________________________________________

Mount drive by running the cell below

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Save the model to your google drive by running the cell below

In [0]:
model_save_name = 'best_model'
path = F"/content/gdrive/My Drive/{model_save_name}" 
model.save(path)

Load the trained model saved in the drive

In [27]:
model.load('/content/gdrive/My Drive/best_model')

Learner(data=ImageDataBunch;

Train: LabelList (300746 items)
x: ImageList
Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64)
y: CategoryList
forward_slash,forward_slash,forward_slash,forward_slash,forward_slash
Path: final_output_images;

Valid: LabelList (75228 items)
x: ImageList
Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64),Image (3, 64, 64)
y: CategoryList
forward_slash,forward_slash,forward_slash,forward_slash,forward_slash
Path: final_output_images;

Test: None, model=Sequential(
  (0): Sequential(
    (0): Sequential(
      (conv0): Conv2d(3, 96, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (norm0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu0): ReLU(inplace=True)
      (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (denseblock1): _DenseBlock(
        (denselayer1): _DenseLayer(
          (norm1): Bat

In [23]:
print(os.listdir('final_output_images/val/X'))


['exp48341.jpg', 'x_109731.jpg', 'x_41334.jpg', 'exp94866.jpg', 'exp51198.jpg', 'x_9909.jpg', 'exp64817.jpg', 'exp38474.jpg', 'exp66321.jpg', 'exp15086.jpg', 'exp49990.jpg', 'x_81812.jpg', 'exp47974.jpg', 'exp9920.jpg', 'x_108504.jpg', 'exp88814.jpg', 'x_113351.jpg', 'exp11698.jpg', 'exp20939.jpg', 'exp60011.jpg', 'exp52688.jpg', 'exp98705.jpg', 'exp78836.jpg', 'x_105886.jpg', 'exp58451.jpg', 'exp72012.jpg', 'x_77738.jpg', 'x_104662.jpg', 'exp105787.jpg', 'x_61782.jpg', 'exp19107.jpg', 'exp35852.jpg', 'exp50636.jpg', 'exp59150.jpg', 'exp24045.jpg', 'x_43032.jpg', 'x_20888.jpg', 'exp14966.jpg', 'exp73501.jpg', 'exp31044.jpg', 'exp60326.jpg', 'exp104974.jpg', 'x_90968.jpg', 'exp43794.jpg', 'exp53700.jpg', 'exp94349.jpg', 'exp103289.jpg', 'x_48865.jpg', 'exp8874.jpg', 'x_91570.jpg', 'exp49365.jpg', 'exp12023.jpg', 'exp5242.jpg', 'exp65904.jpg', 'exp6777.jpg', 'exp96720.jpg', 'x_6511.jpg', 'x_106692.jpg', 'exp4595.jpg', 'X_92529.jpg', 'exp101521.jpg', 'exp2816.jpg', 'exp99250.jpg', 'exp540

In [0]:
pred = model.predict(open_image('segl0w1.jpg'))
print(pred[0])
